# 用户指南，第54章：添加更多文件格式的支持
[原文链接](https://www.music21.org/music21docs/usersGuide/usersGuide_54_extendingConverter.html)

在这个例子中，我们不会使用`from music21 import *`导入music21中的全部内容，而是只导入我们需要的模块。如果你正在开发一个新的`SubConverter`转换器类给自己使用，你可以像往常一样导入全部内容。但是，如果你将来有一天准备把你的代码贡献给music21官方仓库，那么最好是不要使用`import *`，以免导致循环引用。

In [1]:
from music21 import converter, note, stream, meter

这里我们定义一个简单的文件格式，singlebeat格式，后缀名为`.sb`。它只包含由A~G的字母组成的“单词”，以空格分隔。A~G表示音符的音名（不包含升降调）。每个单词中的音符平分一个四分音符的时长。

In [2]:
class SingleBeat(converter.subConverters.SubConverter):
    # 注意，在定义只包含一个元素的元组时，需要在元素后面加上逗号
    registerFormats = ('singlebeat',)  
    registerInputExtensions = ('sb',)  

    # 这里只需定义parseData方法，让父类SubConverter负责关联.sb文件和以.sb结尾的URL

    def parseData(self, strData, number=None):  # movement number is ignored...
        '''  'AB C' -> A-8th, B-8th, C-qtr '''
        strDataList = strData.split()
        s = stream.Part()
        m = meter.TimeSignature('4/4')
        s.insert(0, m)
        for beat in strDataList:
            ql = 1.0/len(beat)
            for n in beat:
                nObj = note.Note(n)
                nObj.duration.quarterLength = ql
                s.append(nObj)
        self.stream = s.makeMeasures()

接下来把我们的转换器注册到converter模块中，并与`.sb`文件关联

In [3]:
converter.registerSubConverter(SingleBeat)

现在我们即可通过converter.parse()解析刚才定义的格式

In [4]:
s = converter.parse('CDC DE F GAGB GE C DEFED C', format='singleBeat')
s.show('text')

{0.0} <music21.stream.Measure 1 offset=0.0>
    {0.0} <music21.clef.TrebleClef>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.note.Note C>
    {0.3333} <music21.note.Note D>
    {0.6667} <music21.note.Note C>
    {1.0} <music21.note.Note D>
    {1.5} <music21.note.Note E>
    {2.0} <music21.note.Note F>
    {3.0} <music21.note.Note G>
    {3.25} <music21.note.Note A>
    {3.5} <music21.note.Note G>
    {3.75} <music21.note.Note B>
{4.0} <music21.stream.Measure 2 offset=4.0>
    {0.0} <music21.note.Note G>
    {0.5} <music21.note.Note E>
    {1.0} <music21.note.Note C>
    {2.0} <music21.note.Note D>
    {2.2} <music21.note.Note E>
    {2.4} <music21.note.Note F>
    {2.6} <music21.note.Note E>
    {2.8} <music21.note.Note D>
    {3.0} <music21.note.Note C>
    {4.0} <music21.bar.Barline type=final>


在解析字符串时，也可以通过`singleBeat:`文件头来指定字符串格式：

In [5]:
s = converter.parse('singleBeat: CDC DE F GAGB GE C DEFED C')
s[-1][0]

<music21.note.Note G>

也可以从外部文件中读入

In [6]:
from music21 import environment
e = environment.Environment()
fp = e.getTempFile('.sb')
with open(fp, 'w') as f:
    f.write('CDC DE F GAGB GE C DEFED C')

print(fp)

C:\Users\lin\AppData\Local\Temp\music21\tmp2p_35zmy.sb


In [7]:
s2 = converter.parse(fp)
s2.show('text')

{0.0} <music21.metadata.Metadata object at 0x2c461ad8b90>
{0.0} <music21.stream.Measure 1 offset=0.0>
    {0.0} <music21.clef.TrebleClef>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.note.Note C>
    {0.3333} <music21.note.Note D>
    {0.6667} <music21.note.Note C>
    {1.0} <music21.note.Note D>
    {1.5} <music21.note.Note E>
    {2.0} <music21.note.Note F>
    {3.0} <music21.note.Note G>
    {3.25} <music21.note.Note A>
    {3.5} <music21.note.Note G>
    {3.75} <music21.note.Note B>
{4.0} <music21.stream.Measure 2 offset=4.0>
    {0.0} <music21.note.Note G>
    {0.5} <music21.note.Note E>
    {1.0} <music21.note.Note C>
    {2.0} <music21.note.Note D>
    {2.2} <music21.note.Note E>
    {2.4} <music21.note.Note F>
    {2.6} <music21.note.Note E>
    {2.8} <music21.note.Note D>
    {3.0} <music21.note.Note C>
    {4.0} <music21.bar.Barline type=final>


如果不确定文件后缀名，那么可以使用`format`参数强制指定解析格式

In [8]:
s3 = converter.parse(fp, format='singleBeat')
s3

<music21.stream.Part 0x2c461821dc0>

现在，在music21中所有支持多种文件格式的地方，singleBeat都会被列为一种受支持的文件格式

In [9]:
from music21 import common
common.findFormat('singleBeat')

('singlebeat', '.sb')

通过`converter.resetSubConverters`，可以让music21忘记我们定义的格式。

In [10]:
converter.resetSubConverters()

现在你可以让music21支持任何格式的转换，是时候回到理解你拥有的音乐了。节拍是一个比简单的 4/4 或 6/8 要深得多的概念。要了解更多关于如何进行节拍分析的内容，请参阅 [用户指南，第55章：高级节拍主题](55.ipynb)。